# IBA Probe for Headroom

In this tutorial we will demonstrate the "headroom" probe that is used to messure headroom traffic or available load on interfaces from source to destination device.

The first thing we need to do to is setup our AOS server and identify the blueprint we want to use.

In [1]:
aos_server = '172.20.91.3'
blueprint_name = "rack-based-blueprint-f03bfadb"

Next let's create a AOS python client that we will use to create our headroom probe

In [2]:
import json
from aospy.swagger import Client

aos = Client('https://' + aos_server)

Now let's use that client to get the blueprint, so we can have the blueprint_id value we will need when we post the headroom probe.

In [3]:
from aospy.swagger.groups.blueprints import Blueprints
bp_index = Blueprints(aos)
bp = bp_index[blueprint_name]

In [4]:
print bp.id

rack-based-blueprint-f03bfadb


Now let's import the headroom probe from the library

In [5]:
from aospy.ibaprobelib.headroom import headroom_probe

Next let's create the headroom probe payload information that we will need.  The headroom probe takes a few parmateres which we can see if we introspect the function:

In [6]:
?headroom_probe

The following paramaters are required:  
`label` - Descriptive name for the probe
`src_node_label` - label of the source server/switch for the headroom calculation
`dst_node_label` - label of the destination server/switch for the headroom calculation

So for the case of reuse, we will define these variables here:

In [7]:
probe_label = 'Ryan-headroom-probe'
src_node_label = 'rack1_server1'
dst_node_label = 'rack2_2_server1'

Notice how `src_node_label` and `dst_node_label` define the source and destination nodes.

We get these from the blueprint.  For exmaple, our demo blueprint:

<img src="media/headroom-1.png">

Now let's create the body data from these variables:

In [8]:
probe_body = headroom_probe(label=probe_label, src_node_label=src_node_label, dst_node_label=dst_node_label)

If you'd like to dump the contents of the probe_body, you can use the following:

In [9]:
# print json.dumps(probe_body, indent=2)

Finally, post the probe body to AOS to create the headroom probe.

In [11]:
probes_url = bp.url + "/probes"

In [12]:
probes_url

u'https://172.20.91.3/api/blueprints/rack-based-blueprint-f03bfadb/probes'

In [13]:
resp = aos.remote.api.post(probes_url, json=probe_body)

In [14]:
resp.ok

True

Now we have our IBA Probe ID:

In [15]:
probe_id = resp.json()['id']

print probe_id

9e821812-3069-4d7f-9e90-5f58591a339a


And now we can see the IBA active probe in the UI:

<img src="media/headroom-2.png">

And then select the probe on the UI to get the details:

<img src="media/headroom-3.png">

# Delete the IBA Headroom Probe

If you want to remove the IBA probe, you can use the API to do so, like the following; just uncomment this out.

In [16]:
resp = aos.remote.api.delete(probes_url + "/%s" % probe_id)
resp.ok

True